In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud
from bs4 import BeautifulSoup

%matplotlib inline
random.seed(42)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\13129\AppData\Roaming\nl

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
questions_data = pd.read_csv('Questions^.csv')
tags_data = pd.read_csv('Tags^.csv')

In [ ]:
questions_data.head()

In [ ]:
tags_data.head()

In [ ]:
questions_data.info()

In [ ]:
tags_data.info()

In [ ]:
tags_residual = tags_data[tags_data.isna().any(axis=1)]
tags_residual.head()

In [ ]:
question_tags_data = pd.merge(questions_data,tags_data,how='inner',on='Id')

In [ ]:
question_tags_data.head()

In [ ]:
question_tags_data.info()

### Dropping na

In [ ]:
question_tags_data.drop(['OwnerUserId','OwnerUserId','Id','CreationDate','Score'],axis=1,inplace=True)
question_tags_data.columns = question_tags_data.columns.str.strip()
question_tags_data.columns = question_tags_data.columns.str.replace(' ', '_')
question_tags_data.columns = question_tags_data.columns.str.lower()

In [ ]:
question_tags_data.head()

In [ ]:
question_tags_data.info()

In [ ]:
question_tags_data[question_tags_data.isna().any(axis=1)].head()

### Text Cleaning

In [ ]:
question_tags_data.dropna(axis=0,inplace=True)
question_tags_data = question_tags_data.reset_index(drop=True)

In [ ]:
question_tags_data.info()

In [ ]:
plt.figure(figsize=(20,3))
count_plot = sns.barplot(x=question_tags_data.tag.value_counts().nlargest(20).index, y=question_tags_data.tag.value_counts().nlargest(20))
count_plot.set_xticklabels(count_plot.get_xticklabels(), rotation=45)

In [ ]:
question_tags_data_grouped = question_tags_data.groupby('tag').head(500).reset_index(drop=True)

In [ ]:
value_counts = question_tags_data_grouped.tag.value_counts()
to_remove = value_counts[value_counts<100].index
question_tags_data_grouped = question_tags_data_grouped[~question_tags_data_grouped.tag.isin(to_remove)]
question_tags_data_grouped = question_tags_data_grouped.reset_index(drop=True)

In [ ]:
print(question_tags_data_grouped.shape)
print(len(question_tags_data_grouped.tag.unique()))
question_tags_data_grouped.head()

In [ ]:
body = []
codes = []
for i in range(question_tags_data_grouped.shape[0]):
    soup = BeautifulSoup(question_tags_data_grouped.body[i])
    to_remove = soup.find_all('code')
    for element in to_remove:
        code = element.extract()
    body.append(soup)
    to_remove = list(str(to_remove))
    code = ''.join(to_remove)
    codes.append(code)
question_tags_data_grouped['body'] = body
question_tags_data_grouped['code'] = codes

In [ ]:
question_tags_data_grouped = question_tags_data_grouped.applymap(lambda text: BeautifulSoup(str(text), 'html.parser').get_text())

In [ ]:
question_tags_data_grouped.head()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
question_tags_data_grouped['title'] = question_tags_data_grouped['title'].apply(lambda x: clean_text(x)) 
question_tags_data_grouped['body'] = question_tags_data_grouped['body'].apply(lambda x: clean_text(x)) 
question_tags_data_grouped['code'] = question_tags_data_grouped['code'].apply(lambda x: clean_text(x)) 

In [ ]:
stop=set(stopwords.words('english'))
question_tags_data_grouped['title_without_stopwords'] = question_tags_data_grouped['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
question_tags_data_grouped['body_without_stopwords'] = question_tags_data_grouped['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
question_tags_data_grouped['code_without_stopwords'] = question_tags_data_grouped['code'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
#Lemmatizing
lemmatizer = WordNetLemmatizer()
def lemmatize_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [ ]:
question_tags_data_grouped['title_without_stopwords'] = question_tags_data_grouped['title_without_stopwords'].apply(lemmatize_sentences)
question_tags_data_grouped['body_without_stopwords'] = question_tags_data_grouped['body_without_stopwords'].apply(lemmatize_sentences)
question_tags_data_grouped['code_without_stopwords'] = question_tags_data_grouped['code_without_stopwords'].apply(lemmatize_sentences)

In [ ]:
print(question_tags_data_grouped['body_without_stopwords'][0])
print(question_tags_data_grouped['title_without_stopwords'][0])
print(question_tags_data_grouped['code_without_stopwords'][0])

In [ ]:
question_tags_data_grouped['title']=question_tags_data_grouped['title_without_stopwords']
question_tags_data_grouped.drop(['title_without_stopwords'],axis=1,inplace=True)
question_tags_data_grouped['body']=question_tags_data_grouped['body_without_stopwords']
question_tags_data_grouped.drop(['body_without_stopwords'],axis=1,inplace=True)
question_tags_data_grouped['code']=question_tags_data_grouped['code_without_stopwords']
question_tags_data_grouped.drop(['code_without_stopwords'],axis=1,inplace=True)
question_tags_data_grouped.to_csv('modified_question_and_body.csv',sep = ',')

In [ ]:
question_tags_data_grouped_2 = question_tags_data_grouped.drop_duplicates(subset='title', keep='first')
question_tags_data_grouped_2.groupby('title').head(10)
print(question_tags_data_grouped.shape, question_tags_data_grouped_2.shape)

In [ ]:
question_tags_data_grouped_2 = question_tags_data_grouped.drop_duplicates(subset='title', keep='first')
question_tags_data_grouped_2.groupby('title').head(10)
print(question_tags_data_grouped.shape, question_tags_data_grouped_2.shape)
